In [1]:
import subprocess
import multiprocessing
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
def run_fssp(instance, init=0, improvement=0, neighbourhood=0):
    try:
        init = "--random-init" if not init else "--srz"
        improvement = "--first" if not improvement else "--best"
        neighbourhood = "--transpose" if neighbourhood == 0 else "--exchange" if neighbourhood == 1 else "--insert"
        command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
        output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
        solution, time = output.decode('utf-8').split('\n')[:2]
        return command, [int(x) for x in solution.split(" ") if x], int(time)
    except Exception as e:
        print(e)
        return command, [], 0


In [3]:
def run_instance(instance):
    res = []
    for init in [0, 1]:
        for improvement in [0, 1]:
            for neighbourhood in [0, 1, 2]:
                res.append(run_fssp(instance, init, improvement, neighbourhood))
    return res


In [4]:
def all_instances_names():
    for filename in os.listdir("../instances"):
        if filename != "bestSolutions.txt":
            yield os.path.join("../instances/", filename)

In [5]:
instances = list(all_instances_names())
pool = multiprocessing.Pool(multiprocessing.cpu_count())
res = list(tqdm(pool.imap_unordered(run_instance, instances), total=len(instances)))        

In [6]:
import itertools
a = list(itertools.chain(*res))

In [7]:
def to_nikita(a):
    times = {}
    for x in a:
        times[x[0].replace("../", "")] = x[2]
    return times

times = to_nikita(a)
times

{'./fssp instances/100_20_01 --random-init --best --exchange': 1845924,
 './fssp instances/100_20_01 --random-init --best --insert': 1983587,
 './fssp instances/100_20_01 --random-init --best --transpose': 2493326,
 './fssp instances/100_20_01 --random-init --first --exchange': 1816290,
 './fssp instances/100_20_01 --random-init --first --insert': 1957238,
 './fssp instances/100_20_01 --random-init --first --transpose': 2469222,
 './fssp instances/100_20_01 --srz --best --exchange': 1845234,
 './fssp instances/100_20_01 --srz --best --insert': 1847052,
 './fssp instances/100_20_01 --srz --best --transpose': 1861380,
 './fssp instances/100_20_01 --srz --first --exchange': 1841685,
 './fssp instances/100_20_01 --srz --first --insert': 1829427,
 './fssp instances/100_20_01 --srz --first --transpose': 1861380,
 './fssp instances/100_20_02 --random-init --best --exchange': 1898868,
 './fssp instances/100_20_02 --random-init --best --insert': 1988245,
 './fssp instances/100_20_02 --random-in

In [ ]:
processed = [
    {
        "instance": s.split()[1].split("/")[-1],
        "init": "srz" if "srz" in s else "random",
        "pivot": "best" if "best" in s else "first",
        "neighborhood": "exchange" if "exchange" in s else ("transpose" if "transpose" in s else "insert"),
        "score": time,
        
    }
    for s, sol, time in a
]
df = pd.DataFrame(processed)

In [ ]:
df